<font size=20>**Using SHAP Values for Model Interpretability**<font>

In [2]:
#!pip install shap


# Imports

In [2]:
import pandas as pd


# Loading Data

In [3]:
df = pd.read_csv('../processed_data/cleaned_data_final.csv')
df.head(5)


,name,male,origin,isMarried,isNoble,popularity,isAlive,season,episode,deaths,episode_num
0,Viserys II Targaryen,1,House Targaryen,0,0,0.605351,0,8,The Bells,Lord Varys - Burned alive by Drogon with his f...,71
1,Walder Frey,1,House Frey,1,1,0.896321,0,6,The Winds of Winter,Lord Walder Frey - Throat slit by Arya Stark w...,60
2,Addison Hill,1,House Lannister,0,1,0.267559,1,0,NaN,NaN,0
3,Aemma Arryn,0,House Arryn,1,1,0.183946,0,5,The Gift,Maester Aemon Targaryen - Died of natural causes.,47
4,Sylva Santagar,0,House Martell,1,1,0.043478,1,0,NaN,NaN,0


In [8]:
X = df
X = X.drop(columns = ["name",'isAlive','episode','deaths','season',
       'episode_num'], axis=1)

y = df[["isAlive"]]

print(f'Features: {X.columns},\n \ny.head(5):\n{y.head()}')


Features: Index(['male', 'origin', 'isMarried', 'isNoble', 'popularity'], dtype='object'),
 
y.head(5):
   isAlive
0        0
1        0
2        1
3        0
4        1
